
###  In this notebook we will clean all the data and put it at on place in sql file. 
    Structure of sql file.
        -- jokes
        -- ratings
        
    We have jokes html file and ratings csv file. We will extract the joke and put it into sql data base as jokes table
    We will put all the ratings into ratings table


In [1]:
import sqlite3 as db

In [2]:
# Connect to a database (or create one if it doesn't exist)
sql_db = 'jester_jokes'
query = 'data/{}.db'.format(sql_db)
conn = db.connect(query)

In [3]:
# Create a 'cursor' for executing commands
c = conn.cursor()
# For any user only these 5 tables will be formed inside whom all the data will be collected. 
c.execute("CREATE TABLE jokes (joke_id INTEGER, joke TEXT)")

In [4]:
schema = "CREATE TABLE ratings(user_id Integer, number_of_jokes_rated Integer,"
for i in range(1, 101):
    schema = schema + 'joke_{} FLOAT(2), '.format(str(i))
schema = schema[:-2] + ')'
c.execute(schema)

    Cleaning HTML file to extract joke data!

In [5]:
import re
import os
import codecs
import pandas as pd
import html2text

In [6]:
file = 'init1.html'
data = codecs.open('data/raw/jokes/' + file, 'r', encoding="cp1252")
joke_html = data.read()

# Extracting joke
joke = html2text.html2text(joke_html)
# Extracting joke_id
joke_id = int(file.split('init')[1].split('.html')[0])

print(joke_id)
print(joke)

1
  
A man visits the doctor. The doctor says "I have bad news for you.You have
cancer and Alzheimer's disease".

The man replies "Well,thank God I don't have cancer!"




In [7]:
# creating a method that input the location of html file and output joke 
def joke_extractor(file):
    data = codecs.open(file, 'r', encoding="cp1252")
    joke_html = data.read()
    joke = html2text.html2text(joke_html)
    joke_id = int(file.split('init')[1].split('.html')[0])
    return (joke_id, joke)

In [8]:
files = os.listdir('data/raw/jokes')
jokes_list = [joke_extractor('data/raw/jokes/'+ file) for file in files if file.endswith('.html') ]

#### Saving jokes_list into sql database!

In [9]:
c.executemany('INSERT INTO jokes VALUES (?,?)', jokes_list)
# Save (commit) the changes
conn.commit()

#### Checking if jokes were saved into database by reading jokes from db

In [10]:
import pandas as pd
import numpy as np
query = 'SELECT * FROM jokes'
jokes_df = pd.read_sql(query, conn)
jokes_df.head()

,joke_id,joke
0,1,"\nA man visits the doctor. The doctor says ""..."
1,10,"| \n\n \nTwo cannibals are eating a clown, o..."
2,100,\n \nQ: Whats the difference between greeti...
3,11,"| \n\n \nQ. What do a hurricane, a tornado, ..."
4,12,| \n\n \nA guy stood over his tee shot for w...


### Saving ratings data into sql table !!
-------------------------------------------------

In [17]:
dataframe = pd.DataFrame()

df1 = pd.read_excel('data/raw/jester-data-1.xls', header = None) 
df2 = pd.read_excel('data/raw/jester-data-2.xls', header = None) 
df3 = pd.read_excel('data/raw/jester-data-3.xls', header = None) 
dataframe =  pd.concat([df1, df2, df3], axis=0)

In [18]:
dataframe.head(2)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07


    Since ratings have no user name or any user identification tag, we we give them user id 

In [19]:
user_id = np.linspace(1, len(dataframe), len(dataframe))
dataframe.insert(0, 'user_id', user_id)
dataframe.head(2)

,user_id,0,1,2,3,4,5,6,7,8,...,91,92,93,94,95,96,97,98,99,100
0,1.0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,2.0,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07


#### Writting query to insert ratings data into sql table

In [23]:
# writting insert query!
query =  'INSERT INTO ratings VALUES('
for i in range(0, 102):
    query = query + '?,'
query = query[:-1] + ')'

In [24]:
# saving dataframe data into sql database
c.executemany(query, dataframe.values)
# Save (commit) the changes
conn.commit()

### Verifying if data had been saved in the sql database!

In [28]:
query = 'SELECT * FROM ratings'
df = pd.read_sql(query, conn)
df.tail()

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,...,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,joke_100
73416,73417,21,99.0,99.0,99.0,99.0,5.78,8.45,99.00,8.20,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
73417,73418,16,99.0,99.0,99.0,99.0,5.24,99.00,99.00,5.68,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
73418,73419,35,99.0,99.0,99.0,99.0,-3.93,99.00,-9.03,-2.67,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
73419,73420,21,99.0,99.0,99.0,99.0,9.42,9.66,99.00,9.17,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
73420,73421,35,99.0,99.0,99.0,99.0,0.19,99.00,-1.50,7.82,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0


#### Also saving a copy of data into csv format !

In [29]:
df.to_csv('data/jester_jokes_rating.csv', index = None)

-----------------------------------------------
                                                        END

-----------------------------------